In [1]:
import numpy as np

In [2]:
with open('movie-xids.npy', 'rb') as f:
    xids = np.load(f, allow_pickle = True)
with open('movie-xmask.npy', 'rb') as f:
    xmask = np.load(f, allow_pickle = True)
with open('movie-labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle = True)

In [3]:
import tensorflow as tf

In [4]:
 dataset = tf.data.Dataset.from_tensor_slices((xids, xmask, labels))

In [5]:
dataset.take(1)

<TakeDataset shapes: ((512,), (512,), (5,)), types: (tf.int32, tf.int32, tf.float64)>

In [6]:
# {input_ids, attention_masks}, outputs

In [7]:
def map_fn(input_ids, masks, labels):
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

In [8]:
dataset = dataset.map(map_fn)

In [9]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (512,), attention_mask: (512,)}, (5,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [10]:
batch_size = 16

In [11]:
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder = True)
dataset.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 5)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [12]:
split = 0.9

size = int((xids.shape[0] / batch_size) * split)

size

479

In [13]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [14]:
tf.data.experimental.save(train_ds, 'train')
tf.data.experimental.save(val_ds, 'val')

In [15]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [16]:
val_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [17]:
train_ds.element_spec == val_ds.element_spec

True

In [18]:
ds = tf.data.experimental.load('train', element_spec = train_ds.element_spec)
ds.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 5)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>